In [1]:
from gidnet_qubit_reuse_algorithm.gidnet_qra import (compile_dynamic_circuit,
                                                     create_qiskit_and_qnet_QAOA_circuits,
                                                     compute_qubit_reuse_sets, 
                                                     generate_dynamic_circuit,)
from biadu_qnet_qubit_reuse.baidu_qnet_qr import (compute_qnet_qubit_reuse_list, 
                                                  from_qiskit_to_qnet,
                                                 compute_qnet_qubit_reuse_list_timing)
from helper_functions.benchmarks import generate_circ
import time

In [2]:
circuit_sizes = [6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50]

qnet_average_time = []
qnet_std_time = []
qnet_average_qubit_width = []



# Set the number of random graph for each qubit number
seed_num = 20

# Set the random shots of the random greedy algorithm
random_shots = 10

# Set the number of QAOA unitary layers
layer_num = 1

for i, q in enumerate(circuit_sizes):
    print("circuit", i+1, "running")
    # create temporary data holders for intermediate results

    tmp_qnet_average_time = []
    tmp_qnet_std_time = []
    tmp_qnet_qubit_width = []
    
    qiskit_circuits, qnet_circuits = create_qiskit_and_qnet_QAOA_circuits(q, seed_num, layer_num)
    
    for qiskit_circuit, qnet_circuit in zip(qiskit_circuits, qnet_circuits):
        qnet_time = %timeit -o compile_dynamic_circuit(qiskit_circuit, qnet_circuit=qnet_circuit, algorithm_type="qnet", qnet_method="random", qnet_shots=random_shots)
        compiled_qnet_circuit = compile_dynamic_circuit(qiskit_circuit, qnet_circuit=qnet_circuit, algorithm_type="qnet", qnet_method="random", qnet_shots=random_shots)

        tmp_qnet_average_time.append(qnet_time.average)
        tmp_qnet_std_time.append(qnet_time.stdev)
        tmp_qnet_qubit_width.append(compiled_qnet_circuit.num_qubits)


    # Data colation
    import numpy as np
    qnet_average_time.append(tmp_qnet_average_time)
    qnet_std_time.append(tmp_qnet_std_time)
    qnet_average_qubit_width.append(tmp_qnet_qubit_width)
        


circuit 1 running
18.4 ms ± 4.66 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
18.8 ms ± 4.51 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
20.2 ms ± 6.38 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
19.8 ms ± 5.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
20.3 ms ± 4.98 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
19.1 ms ± 5.25 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
20.3 ms ± 5.24 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
19 ms ± 4.84 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
19.9 ms ± 5.08 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
20.3 ms ± 5.61 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
19.8 ms ± 5.16 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
19.8 ms ± 5.12 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
19.2 ms ± 4.55 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
18.3 ms ± 5.34 ms per loop (mean ± std. dev. of 

In [3]:
import pandas as pd
# Convert lists to a dictionary
data = {
    'Circuit Sizes': circuit_sizes,
    'QNET Average Runtime': qnet_average_time,  # Added comma
    'QNET Stdev Runtime': qnet_std_time,  # Added comma
    'QNET Width': qnet_average_qubit_width,
}

# Assuming pandas is imported as pd
import pandas as pd

# Convert dictionary to DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV
df.to_csv('data_qaoa_cct/QNET_width_and_runtime_qaoa_data_complete.csv', index=False)


In [4]:
df

,Circuit Sizes,QNET Average Runtime,QNET Stdev Runtime,QNET Width
0,6,"[0.018370334618571342, 0.01882407725571446, 0....","[0.004656355834525185, 0.004506558294539112, 0...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 4, ..."
1,10,"[0.041127457185713666, 0.04204544918571368, 0....","[0.002725545394033881, 0.005288478940615063, 0...","[6, 5, 5, 5, 6, 6, 4, 4, 5, 6, 4, 5, 5, 6, 6, ..."
2,14,"[0.13638052125714598, 0.13669380165714368, 0.1...","[0.004374484091290346, 0.0037954283526995987, ...","[7, 7, 6, 5, 5, 6, 7, 7, 7, 6, 6, 7, 6, 7, 6, ..."
3,18,"[0.43455385999998597, 0.346560837571425, 0.422...","[0.017813731649256127, 0.013377502361989011, 0...","[8, 10, 8, 7, 8, 7, 7, 6, 8, 8, 7, 6, 7, 7, 8,..."
4,22,"[0.9710650325714596, 1.1318133688571703, 0.996...","[0.01447144978490124, 0.02038079065487241, 0.0...","[9, 8, 8, 9, 8, 9, 7, 9, 9, 8, 8, 8, 8, 10, 8,..."
5,26,"[2.0859384791427993, 2.047692890999997, 1.9667...","[0.04250459567621376, 0.020365257385309563, 0....","[10, 11, 11, 9, 11, 8, 9, 9, 11, 9, 8, 9, 10, ..."
6,30,"[3.9197006444284983, 4.0874965684285405, 4.307...","[0.10987846996837684, 0.09939303146468788, 0.1...","[12, 10, 10, 9, 9, 11, 10, 10, 9, 10, 11, 12, ..."
7,34,"[7.721275640571548, 7.822222285143003, 7.25498...","[0.1465415700841026, 0.07343457093852498, 0.14...","[11, 10, 12, 11, 13, 11, 12, 10, 10, 10, 12, 1..."
8,38,"[12.668493911571463, 13.056348711857027, 12.61...","[0.08419044422038843, 0.13218206862590884, 0.1...","[11, 11, 12, 14, 11, 13, 10, 11, 11, 12, 13, 1..."
9,42,"[21.30943124557169, 20.877472902714153, 19.824...","[0.3239378539584638, 0.255930146288709, 0.2904...","[11, 12, 12, 14, 12, 14, 11, 14, 13, 14, 13, 1..."
